<center><font size="50" color="#13347d" >Proyecto de desarrollo de sistema de IA basado en alquileres de pisos – Tercera fase </font></center>

In [1]:
#!pip install --upgrade --force-reinstall transformers datasets numpy scipy gensim   #Dejamos ejecutado en la primera vez este comando para no tener problemas con las dependencias de numpy,gensim y demas
# Empezaremos importando algunas de las librerias
import pandas as pd
import numpy as np
import re
import string

# Importamos el procesamiento de lenguaje natural en español
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# Importamos sklearn para trabajar con los modelos
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<font size="5" color="#3c4d54" >A continuacion empezaremos la tercera fase del proyecto con las partes que pueden servirnos de manera general en lo que queda de ejercicio: La carga del dataset, la limpieza del mismo y su posterior preparacion. </font>


In [2]:
#Montamos la unidad para trabajar en colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import kagglehub

# Descargamos el dataset de la pagina de kaggle.
path = kagglehub.dataset_download("fjcob1/idealista-madrid")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/idealista-madrid


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [5]:
import shutil
destino_drive = '/content/drive/MyDrive/Colab Notebooks' # Ponemos la ruta del drive.
shutil.move(path, destino_drive)

Error: Destination path '/content/drive/MyDrive/Colab Notebooks/idealista-madrid' already exists

In [6]:
data_dir= "/content/drive/MyDrive/Colab Notebooks/2"

In [7]:
df = pd.read_csv(f"{data_dir}/Datos.csv")

# Carga del dataset y vista previa de los primeros 5 registros.
df.head()

,provincia,zona,titulo,PrecioActual,PrecioAnterior,metros,habitaciones,ascensor,localizacion,planta,baños,tags,descripcion,Enlace
0,madrid,ciudad-lineal,"Piso en calle de San Marcelo, 22, Ventas, Madrid",355000,0,69,2.0,S,EXTERIOR,5ª,1,"VIVIENDA,LUMINOSO,VISTAS,REFORMADA,PORTERO,EXT...",Particular Vende vivienda totalmente reformada...,https://www.idealista.com/inmueble/106956987/
1,madrid,carabanchel,"Piso en calle Cabo Nicolás Mur, San Isidro, Ma...",149000,159000,91,3.0,N,EXTERIOR,1ª,0,"PISO,EXCLUSIVA,INMOBILIARIA,OPORTUNIDAD",Inmobiliarias Encuentro vende en exclusiva la ...,https://www.idealista.com/inmueble/106906044/
2,madrid,centro,"Piso en calle de Rodas, Lavapiés-Embajadores, ...",195000,0,36,1.0,S,NaN,2ª,0,"EXCLUSIVA,ESTUDIO",ESTUDIO EN PLENO CENTRO DE MADRIDSarago Servic...,https://www.idealista.com/inmueble/107306175/
3,madrid,usera,"Piso en calle de Ferroviarios, Almendrales, Ma...",195000,0,58,1.0,S,INTERIOR,BAJO,0,"VIVIENDA,HOGAR,FUNCIONAL","Esta acogedora vivienda, ubicada en una planta...",https://www.idealista.com/inmueble/106325171/
4,madrid,tetuan,"Dúplex en Bellas Vistas, Madrid",715000,750000,140,3.0,S,EXTERIOR,2ª,0,"TERRAZA,EXCLUSIVA,MODERNO","Maravilloso ATICO de reciente construcción, co...",https://www.idealista.com/inmueble/106627265/


In [8]:
# Vemos algunos parametros del dataset.
df.describe()


,PrecioActual,PrecioAnterior,metros,habitaciones,baños
count,1.182600e+04,1.182600e+04,11826.000000,11460.000000,11826.000000
mean,1.030501e+06,7.359704e+04,153.790039,2.847731,0.394047
std,1.237718e+06,3.639753e+05,766.217750,1.432402,0.882134
min,1.200000e+04,0.000000e+00,11.000000,1.000000,0.000000
25%,2.890000e+05,0.000000e+00,68.000000,2.000000,0.000000
50%,6.200000e+05,0.000000e+00,103.000000,3.000000,0.000000
75%,1.329000e+06,0.000000e+00,160.000000,3.000000,0.000000
max,2.300000e+07,8.450000e+06,75000.000000,20.000000,7.000000


In [9]:
#Informaciom de los tipos de dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11826 entries, 0 to 11825
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   provincia       11826 non-null  object 
 1   zona            11826 non-null  object 
 2   titulo          11826 non-null  object 
 3   PrecioActual    11826 non-null  int64  
 4   PrecioAnterior  11826 non-null  int64  
 5   metros          11826 non-null  int64  
 6   habitaciones    11460 non-null  float64
 7   ascensor        11033 non-null  object 
 8   localizacion    10730 non-null  object 
 9   planta          10601 non-null  object 
 10  baños           11826 non-null  int64  
 11  tags            11664 non-null  object 
 12  descripcion     11761 non-null  object 
 13  Enlace          11826 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 1.3+ MB


In [10]:
print("Forma inicial del dataset:", df.shape)
print("\nColumnas disponibles:\n", df.columns)
print("\nValores nulos por columna:\n", df.isnull().sum())

Forma inicial del dataset: (11826, 14)

Columnas disponibles:
 Index(['provincia', 'zona', 'titulo', 'PrecioActual', 'PrecioAnterior',
       'metros', 'habitaciones', 'ascensor', 'localizacion', 'planta', 'baños',
       'tags', 'descripcion', 'Enlace'],
      dtype='object')

Valores nulos por columna:
 provincia            0
zona                 0
titulo               0
PrecioActual         0
PrecioAnterior       0
metros               0
habitaciones       366
ascensor           793
localizacion      1096
planta            1225
baños                0
tags               162
descripcion         65
Enlace               0
dtype: int64


In [11]:
df = df[['descripcion', 'PrecioActual']].copy()
df.dropna(subset=['descripcion', 'PrecioActual'], inplace=True)
# Eliminar duplicados en la descripción
df.drop_duplicates(subset='descripcion', inplace=True)
# Eliminamos los outliers
df=df[(df['PrecioActual']>=100000) & (df['PrecioActual'] <=350000)]
#Reseteamos el indice
df.reset_index(drop=True, inplace=True)
# Resultado
print("Dataset limpio:")
print(df.shape)
df.head()

Dataset limpio:
(3304, 2)


,descripcion,PrecioActual
0,Inmobiliarias Encuentro vende en exclusiva la ...,149000
1,ESTUDIO EN PLENO CENTRO DE MADRIDSarago Servic...,195000
2,"Esta acogedora vivienda, ubicada en una planta...",195000
3,"Negociable, incluir los muebles que figuran en...",275000
4,Ubicado a tan solo 200 metros de la estación d...,220000


<font size="5" color="#3c4d54" >Despues de ver el dataset y analizar las partes mas importantes del mismo de cara a nuestro proyecto , pasamos a la parte del entrenamiento del dataset para realizar el preprocesado de los datos. </font>


In [12]:
# Usaremos esta division de entrenamiento para todos los modelos.
X_train, X_test, y_train, y_test = train_test_split(df['descripcion'], df['PrecioActual'], test_size=0.2, random_state=42,shuffle=True)

<font size="5" color="#3c4d54" >Empezamos ya con la creacion de cada uno de los modelos. </font>


<font size="50" color="#00000" >1.</font><font size="6" color="#246a87" >Método tradicional: Frecuencia de Término – Frecuencia Inversa de Documento (TF-IDF) en combinación con un modelo de aprendizaje automático (regresión   basada en máquina de soporte vectorial o regresión lineal).  </font>


In [13]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
#Primera parte preprocesado de los datos. Reutilizaremos una funcion del notebook de nlp tradicional adaptandola a lo que queremos.
def preprocess_text(text):

  text = text.lower()
  text = re.sub(r'[^a-z\s]', ' ', text)
  #print(f"Esto es el código sin números y caracteres especiales: {text}")


  words = word_tokenize(text)
  #print(f"Esto es el código tokenizado: {words}")

  # Stopwords

  stop_words = set(stopwords.words('spanish'))

  words = [word for word in words if word not in stop_words]

  #print(f"Esto es el código sin stopwords: {words}")
  # Lemmatization

  lemmatizer = WordNetLemmatizer()

  words_lemmatized = [lemmatizer.lemmatize(word) for word in words]
  #print(f"Esto es el código con lemmatization: {words_lemmatized}")

  text_final = ' '.join(words_lemmatized)
  #text = re.sub(r'  ', ' ', text)
  #print(f"Esto es el text final: {text_final}")
  return text_final

In [14]:
# Imprimimos la primera fila para ver que no esta procesada.
print(X_train.iloc[0])

PRIMER NIVEL INMOBILIARIA VENDE en exclusiva estupenda vivienda en el barrio Carabanchel Bajo, con todos los servicios próximos.LA VIVIENDA SE ENCUENTRA ALQUILADA, CON CONTRATO DE ALQUILER VIGENTE.Su distribución es la siguiente: Hall de entrada, sala de estar o habitación, cuarto de baño con bañera, salón distribu


In [15]:
import nltk #Importamos libreria nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
X_train_limpio = [preprocess_text(text) for text in X_train]
X_test_limpio = [preprocess_text(text) for text in X_test]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# Vemos la fila procesada.
print(X_train_limpio[0])

primer nivel inmobiliaria vende exclusiva estupenda vivienda barrio carabanchel bajo servicios pr ximos vivienda encuentra alquilada contrato alquiler vigente distribuci n siguiente hall entrada sala habitaci n cuarto ba ba sal n distribu


In [17]:
# Usamos tfid para vectorizar. le metemos tambien el stopwords en el vectorizador
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=stopwords.words('spanish'),
    min_df=5,
    max_df=0.8
)


X_trainVectorizada = vectorizer.fit_transform(X_train_limpio)
X_testVectorizada = vectorizer.transform(X_test_limpio)

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Definimos los posibles valores de alpha para Ridge
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

# Creamos el modelo base
ridge = Ridge()

# GridSearch con validación cruzada
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')

# Entrenamos sobre los datos vectorizados
grid_search.fit(X_trainVectorizada, y_train)

# Obtenemos el mejor modelo
best_ridge = grid_search.best_estimator_
print("Mejor alpha:", grid_search.best_params_['alpha'])

Mejor alpha: 1.0


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Predicciones con el mejor modelo encontrado
y_pred = best_ridge.predict(X_testVectorizada)


maePrimerModelo = mean_absolute_error(y_test, y_pred)
rmsePrimerModelo = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"✅ MAE: {maePrimerModelo:.2f}")
print(f"✅ RMSE: {rmsePrimerModelo:.2f}")

MAE: 46398.20
RMSE: 56000.74


<font size="50" color="#00000" >2.</font><font size="6" color="#246a87" >Modelo basado en Word embeddings: un modelo Word embedding como Word2Vec, junto con redes neuronales convolucionales (CNNs).  </font>

<font size="5" color="#3c4d54" >Para este segundo modelo no nos interesa usar la funcion de preprocesado de antes. En el caso de Word2Vec nos interesa utilizar palabras naturales tal cual, ya que interesa el contexto de la frase. Quitariamos la parte del stompwords y el lemmatizer y dejariamos la conversion a minusculas, la eliminacion de los caracteres especiales y por último y en este caso lo mas importante de este preprocesado, la tokenizacion.  </font>


In [23]:
!pip install spacy  # Instalar spaCy, Lo ejecutamos lo primero , si no da error luego el transformer
!python -m spacy download es_core_news_sm

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 69.5 MB/s eta 0:00:00
✔ Download and installation s

In [20]:
import re
import spacy

# Cargar el modelo de español de spaCy
nlp = spacy.load("es_core_news_sm")

def preprocess_text1(text):
    # Convertir el texto a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos (pero mantener letras con tilde y ñ ya que trabajamos idioma español)
    text = re.sub(r'[^a-záéíóúüñ\s]', ' ', text)

    # Procesar el texto con spaCy
    doc = nlp(text)

    # Tokenizar y filtrar tokens
    tokens = [token.text for token in doc if not token.is_space]

    return tokens

In [21]:
#Preprocesamos xtrainy y xtest

X_train_limpio = [preprocess_text1(text) for text in X_train]
X_test_limpio = [preprocess_text1(text) for text in X_test]

In [22]:
# Imprimimos xtrainlimpio
print(X_train_limpio[0])

['primer', 'nivel', 'inmobiliaria', 'vende', 'en', 'exclusiva', 'estupenda', 'vivienda', 'en', 'el', 'barrio', 'carabanchel', 'bajo', 'con', 'todos', 'los', 'servicios', 'próximos', 'la', 'vivienda', 'se', 'encuentra', 'alquilada', 'con', 'contrato', 'de', 'alquiler', 'vigente', 'su', 'distribución', 'es', 'la', 'siguiente', 'hall', 'de', 'entrada', 'sala', 'de', 'estar', 'o', 'habitación', 'cuarto', 'de', 'baño', 'con', 'bañera', 'salón', 'distribu']


In [24]:
# 2. Cargar el modelo Word2Vec SBWCE preentrenado, tenemos subido el modelo a google colab
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format('/content/sbw_vectors.bin', binary=True)
vector_dim = w2v_model.vector_size

In [25]:
# Convertimos palabras a numeros

all_tokens = set(token for doc in X_train_limpio for token in doc)
word_index = {word: idx + 1 for idx, word in enumerate(all_tokens)}  # +1 para reservar el 0 (padding)

def tokens_to_indices(tokens):
    return [word_index[token] for token in tokens if token in word_index]

# Aplicar a ambos conjuntos
X_train_seq = [tokens_to_indices(tokens) for tokens in X_train_limpio]
X_test_seq = [tokens_to_indices(tokens) for tokens in X_test_limpio]

In [26]:
#Rellenamos con ceros para tener la misma longitud
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 150

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')


In [27]:
# 9. Y utilizamos la matriz de embeding con nuestro modelo preentrenado
embedding_matrix = np.zeros((len(word_index)+1, vector_dim))
for word, idx in word_index.items():
    if word in w2v_model:
        embedding_matrix[idx] = w2v_model[word]


In [28]:
# Utilizamos un modelo sequential y le añadimos la capa de  embedding unido a la capa inicial de la red y y despues la cnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
# 10. MODELO CNN. Inicialmente probamos con un lstm pero el resultado de la mae y rsme no era ni parecido al del primer modelo. Decidimos cambiar este para mejorar los resultados
model = Sequential()
model.add(Embedding(
    input_dim=embedding_matrix.shape[0],
    output_dim=vector_dim,
    weights=[embedding_matrix],
    input_length=MAX_LEN,
    trainable=True
))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(Conv1D(64, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:

# 11. Añadimos earlystopping , esto nos sirve para mejorar el resultado de la mae.
early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)

# 12. Entrenamiento con 20 epocas para mejorar el modelo
model.fit(
    X_train_pad,        # Entrada ya tokenizada y con padding
    y_train,            # Precio
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 15s 156ms/step - loss: 226923.3281 - mae: 226923.3281 - val_loss: 99904.8281 - val_mae: 99904.8281
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - loss: 72555.9922 - mae: 72555.9922 - val_loss: 62589.5859 - val_mae: 62589.5859
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 150ms/step - loss: 63191.0781 - mae: 63191.0781 - val_loss: 60606.2109 - val_mae: 60606.2109
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 151ms/step - loss: 60696.4180 - mae: 60696.4180 - val_loss: 59591.7617 - val_mae: 59591.7617
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 143ms/step - loss: 58492.6133 - mae: 58492.6133 - val_loss: 61296.8164 - val_mae: 61296.8164
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 22s 160ms/step - loss: 60495.9258 - mae: 60495.9258 - val_loss: 59205.4688 - val_mae: 59205.4688
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - loss: 58452.4297 - mae: 58452.4297 - val_loss: 58861.9648 - val_mae: 58861.9648
Epoch 8/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step 

In [30]:
# 13. Realizamos la predicion y obtemenos las metricas.
y_pred = model.predict(X_test_pad).flatten()

maeSegundoModelo = mean_absolute_error(y_test, y_pred)
rmseSegundoModelo = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"✅ MAE : {maeSegundoModelo:.2f} €")
print(f"✅ RMSE : {rmseSegundoModelo:.2f} €")

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
✅ MAE : 53394.39 €
✅ RMSE : 63974.79 €


<font size="50" color="#00000" >3.</font><font size="6" color="#246a87" >Modelo basado en Transformers: entrenamiento mediante fine-tuning de un modelo preentrenado.  </font>


In [31]:
# Importacion de las herramientas del transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [32]:
# Modelo de roberta en español
model_name = "PlanTL-GOB-ES/roberta-base-bne"

In [33]:
# Nos apoyamos de la pagina https://huggingface.co/docs/transformers/es/training para realizar la carga del modelo preentrenado con su trainer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Funcion de preprocesado. Solamente aplicamos el strip ya que los transformers no requieren de hacer un preprocesado del texto tan elevado como los otros enfoques
def preprocess_text2(text):
    return str(text).strip()

In [35]:
# Aplicamos el preprocesamiento
X_train_limpio = X_train.apply(preprocess_text2)
X_test_limpio = X_test.apply(preprocess_text2)

In [36]:

from datasets import Dataset
import pandas as pd

# Creamos un dataframe con los textos y las etiquetas para el trairner.
train_df = pd.DataFrame({'text': X_train_limpio, 'label': y_train})
test_df = pd.DataFrame({'text': X_test_limpio, 'label': y_test})
train_df["label"] = train_df["label"].astype("float32")
test_df["label"] = test_df["label"].astype("float32")

# Convertimos a formato Dataset de HuggingFace
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Realizamos la tokenizacion
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2643 [00:00<?, ? examples/s]

Map:   0%|          | 0/661 [00:00<?, ? examples/s]

In [37]:
!pip install evaluate
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

# Métrica personalizada (MAE) y rsme que la obtenemos manualmente ya que huggin no lo permite
mae_metric = evaluate.load("mae")
# Funcion para las evaluaciones
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.flatten()

    mae = mae_metric.compute(predictions=predictions, references=labels)["mae"]
    rmse = np.sqrt(np.mean((predictions - labels) ** 2))

    return {
        "mae": mae,
        "rmse": rmse
    }

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    metric_for_best_model="mae",
    report_to=[]
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
# Entrenamos el modelo. Aqui observamos los tiempos mas largos de entrenamiento. Imprescindible usar gpu de colab para cargar el trainer. Si no el proceso puede durar mucho tiempo
trainer.train()

Step,Training Loss


In [ ]:
# Obtenemos las prediciones
predictions = trainer.predict(test_tokenized)
y_pred = predictions.predictions.flatten()

# Etiquetas alineadas
y_true = np.array(y_test)

# Métricas
maeTercerModelo = mean_absolute_error(y_true, y_pred)
rmseTercerModelo = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"✅ MAE (Error absoluto medio): {maeTercerModelo:.2f} €")
print(f"✅ RMSE (Raíz del error cuadrático medio): {rmseTercerModelo:.2f} €")

<font size="50" color="#00000" >4.</font><font size="6" color="#246a87" >Evaluación de los modelos </font>

<font size="6" color="#083d52" >PRIMER MODELO</font>

In [ ]:
print(f"✅ MAE: {maePrimerModelo:.2f}")
print(f"✅ RMSE: {rmsePrimerModelo:.2f}")


<font size="6" color="#083d52" >SEGUNDO MODELO</font>

In [ ]:
print(f"✅ MAE : {maeSegundoModelo:.2f} €")
print(f"✅ RMSE : {rmseSegundoModelo:.2f} €")


<font size="6" color="#083d52" >TERCER MODELO</font>

In [ ]:
print(f"✅ MAE (Error absoluto medio): {maeTercerModelo:.2f} €")
print(f"✅ RMSE (Raíz del error cuadrático medio): {rmseTercerModelo:.2f} €")

<font size="5" color="#3c4d54" >Como podemos observar los primeros dos modelos ofrecen unas metricas mas adecuadas que el ultimo modelo. En este caso intentamos cambiar en el segundo modelo la red cnnn por una lstm pero los resultados de la mae se iban hasta los 200.000 distando mucho del resultado del primer modelo y contando con que el segundo al estar mas preparado debería haber ofrecido mejores resultados.
En el caso del ultimo modelo , observamos unos tiempos mucho mas altos en el entrenamiento del modelo y una perdida bastante alta. Las metricas del transformer salen notablemente mas altas . Entiendo que se debe a que los transformers requieren de millones de datos y nuestro dataset no tiene tanta informacion para el modelo lo que quiza nos este provocando un sobreajuste.</font>
